In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


CRIM - уровень преступности на душу населения
ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
INDUS - доля акров неторгового бизнеса на город.
CHAS - 1, если граничит с рекой, 0 иначе
NOX - концентрация оксидов азота (частей на 10 миллионов)
RM - среднее количество комнат в квартире
AGE - доля жилых домов, построенных до 1940 г.
DIS - взвешенные расстояния до пяти центров занятости Бостона
RAD - индекс доступности к радиальным магистралям
TAX - полная ставка налога на имущество за 10 000 долларов США
PTRATIO - соотношение учеников и учителей по городам
B - 1000(Bk - 0.63)^2, где Bk доля чернокожих людей по городам
LSTAT - % более низкого статуса населения
MEDV - Средняя стоимость домов в 1000 долларов.

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [24]:
pd.read_sql(
    """
    SELECT SUM(CRIM IS NULL) CRIM_NULL,
       SUM(ZN IS NULL) ZN_NULL,
       SUM(INDUS IS NULL) INDUS_NULL,
       SUM(CHAS IS NULL) CHAS_NULL,
       SUM(NOX IS NULL) NOX_NULL
    FROM boston
   """,
    con,
)

,CRIM_NULL,ZN_NULL,INDUS_NULL,CHAS_NULL,NOX_NULL
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [25]:
pd.read_sql(
    """
    SELECT COUNT(DISTINCT CRIM) CRIM,
        COUNT(DISTINCT ZN) ZN,
        COUNT(DISTINCT INDUS) INDUS,
        COUNT(DISTINCT CHAS) CHAS,
        COUNT(DISTINCT NOX) NOX
    FROM boston

    """,
    con,
)



,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
a= pd.read_sql(
    """
select * from 
((SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (select COUNT(CRIM)/2  FROM boston )),
(SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (select COUNT(ZN)/2  FROM boston )),
(SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (select COUNT(INDUS)/2  FROM boston )),
(SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (select COUNT(CHAS)/2  FROM boston )),
(SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (select COUNT(NOX)/2  FROM boston )));
    """,
    con,
)
print(a)


      CRIM   ZN  INDUS  CHAS    NOX
0  0.25915  0.0   9.69   0.0  0.538


In [10]:
b = pd.read_sql(
    """
SELECT MIN(CRIM),
MIN(ZN),
MIN(INDUS),
MIN(CHAS),
MIN(NOX)
 FROM boston
 
    """,
    con,
)
print(b)

   MIN(CRIM)  MIN(ZN)  MIN(INDUS)  MIN(CHAS)  MIN(NOX)
0    0.00632      0.0        0.46        0.0     0.385


In [12]:
i = 0
cols = a.columns
for i in range(4):
    c = a.iat[0,i]-b.iat[0,i]
    if c == 0:
        print(cols[i])
    i+=1



ZN
CHAS


4. Выведите разницу между средним количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [70]:
num_houses = [25, 50, 100, 200, 300]

def room_max_price_house():
    a = pd.read_sql(
            """
        SELECT AVG(RM) as roomAvg FROM boston WHERE MEDV = (SELECT MAX(MEDV) from boston)
                
            """,
            con,
        )
    return round(a.iat[0,0],2)

def room_meed_price_house(n):        
    b = f'SELECT AVG(RM) as roomAvg FROM (SELECT RM FROM BOSTON order by MEDV asc limit {n})'
    return round(pd.read_sql(b, con).iat[0,0], 2)

    
i=0
for i in num_houses:
    print(f'Количество комнат в самой дорогой недвижимости: {room_max_price_house()}, по {i} домам с минимальной ценой: {room_meed_price_house(i)}, разница: {round(room_max_price_house()-room_meed_price_house(i),2)}') 
    i+=1

Количество комнат в самой дорогой недвижимости: 7.48, по 25 домам с минимальной ценой: 5.75, разница: 1.73
Количество комнат в самой дорогой недвижимости: 7.48, по 50 домам с минимальной ценой: 5.75, разница: 1.73
Количество комнат в самой дорогой недвижимости: 7.48, по 100 домам с минимальной ценой: 5.89, разница: 1.59
Количество комнат в самой дорогой недвижимости: 7.48, по 200 домам с минимальной ценой: 5.91, разница: 1.57
Количество комнат в самой дорогой недвижимости: 7.48, по 300 домам с минимальной ценой: 5.97, разница: 1.51


Среднее количество комнат в самой дорогой недвижимости составило 7,48. По домам с минимальной стоимостью количество комнат варьировалось от 5,75 до 5,97.
Т.о. в дорогой недвидимости количество комнат в среднем выше - на 1,51 - 1,73 комнаты.
При этом в сравнении с выборкой от 25 до 300 домов разница по минимальным ценам составляла незначительные показатели не более (1,73-1,51) 0,22.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [74]:
pd.read_sql(
    """
    SELECT LSTAT, rank FROM (
    SELECT LSTAT, MEDV, RANK() OVER (order by LSTAT asc) as rank FROM boston)
    WHERE MEDV = (SELECT MAX(MEDV) from boston)
    """,
    con,
)

,LSTAT,rank
0,1.73,1
1,1.92,2
2,2.88,6
3,2.96,8
4,2.97,9
5,3.16,15
6,3.26,17
7,3.32,18
8,3.70,26
9,3.73,27


Владельцами наиболее дорогой недвижимости являютсялюди с более высоким статусом.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [77]:
pd.read_sql(
    """
    SELECT CHAS as RIVER, round(AVG(MEDV),2) as avg_price_houses FROM boston
    GROUP BY CHAS 
    """,
    con,
)



,RIVER,avg_price_houses
0,0.0,22.09
1,1.0,28.44


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [82]:
a = pd.read_sql(
    """
    select CRIM, ZN, INDUS, NOX from(
    select ((SELECT AVG(CRIM)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(CRIM)   FROM boston WHERE CHAS = 0)) as CRIM
    , ((SELECT AVG(ZN)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(ZN)   FROM boston WHERE CHAS = 0)) as ZN
    , ((SELECT AVG(INDUS)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(INDUS)   FROM boston WHERE CHAS = 0)) as INDUS
    , ((SELECT AVG(NOX)   FROM boston WHERE CHAS = 1) - 
    (SELECT AVG(NOX)   FROM boston WHERE CHAS = 0)) as NOX)
    """,
    con,
)   

i = 0
cols = a.columns
for i in range(4):
    b = a.iat[0,i]
    if b > 0:
        print(cols[i])
    i+=1

INDUS
NOX


Если дом граничит с рекой, то среднее значение выше по показателям: доля акров неторгового бизнеса на город и концентрация оксидов азота (частей на 10 миллионов).
При этом обратная ситуация для таких показателей как уровень преступности на душу населения и доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [121]:
perc_NOX = pd.read_sql(
        """
            SELECT percentile, MAX(NOX) as NOX
            FROM (SELECT NOX, NTILE(10) OVER (ORDER BY NOX) AS percentile FROM boston)
            GROUP BY percentile
            """,
        con
    )

perc_INDUS = pd.read_sql(
    """
        SELECT percentile, MAX(INDUS) as INDUS
        FROM (SELECT INDUS, NTILE(10) OVER (ORDER BY INDUS) AS percentile FROM boston)
        GROUP BY percentile
        """,
        con
    )
perc_NOX_INDUS = perc_NOX.set_index('percentile').join(perc_INDUS.set_index('percentile')) 
     
print(perc_NOX_INDUS) 

              NOX  INDUS
percentile              
1           0.426   2.89
2           0.442   4.39
3           0.472   5.96
4           0.507   7.38
5           0.538   9.69
6           0.580  13.89
7           0.609  18.10
8           0.671  18.10
9           0.713  19.58
10          0.871  27.74


Взаимосвязь между показателями присутсвует: чем выше значения долей промышленной застройки (INDUS), тем выше концентрация оксидов азота(NOX)